# Importing Data

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import scipy 
import statsmodels.api as sm 
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime as dt
import yfinance as yf

In [2]:
us_data_path = '/Users/talhajamal/Documents/Documents/Imperial/Courses/Semester 2/Empirical Finance/IndividualProject/coursework_1/Data_coursework_1.xlsx'
us_data = pd.read_excel(us_data_path, sheet_name='US')
us_data['Date'] = pd.to_datetime(us_data['Date'], dayfirst=True)
#start_date = dt(1984, 1, 1)
#us_data = us_data[us_data['Date'] > start_date] # Last 40 years of data

In [3]:
# Summary Statistics for US Data
us_data["Simple Returns"] = us_data['Stock Index'].pct_change()
us_data["Log Returns"] = np.log(us_data['Stock Index'] / us_data['Stock Index'].shift(1))
us_data["ST Returns"] = us_data['Short-term Yield'].pct_change()
us_data["LT Returns"] = us_data['Long-term Yield'].pct_change()
us_data['xt_1'] = us_data['Long-term Yield'].shift(1) - us_data['Short-term Yield'].shift(1)
us_data.dropna(inplace=True)
us_data.reset_index(drop = True, inplace=True)
us_data.head()

,Date,Stock Index,Short-term Yield,Long-term Yield,Simple Returns,Log Returns,ST Returns,LT Returns,xt_1
0,1792-02-29,2.496140,4.198,4.198,0.062644,0.060760,0.061173,0.061173,0.0
1,1792-03-31,2.297596,4.940,4.940,-0.079541,-0.082882,0.176751,0.176751,0.0
2,1792-04-30,2.350072,5.143,5.143,0.022840,0.022583,0.041093,0.041093,0.0
3,1792-05-31,2.562141,4.557,4.557,0.090239,0.086397,-0.113941,-0.113941,0.0
4,1792-06-30,2.512370,4.675,4.675,-0.019426,-0.019617,0.025894,0.025894,0.0


# Expanding Window Regression

In [5]:
# Expanding Window Regression
expanding_model_prediction = []
expanding_benchmark_prediction = []
expanding_actual = []

for i in range(120, len(us_data) - 1, 1): #2784
    #setup data
    x_train = us_data[['xt_1']][:i]
    y_train = us_data[['Log Returns']][:i]
    # Run Regression
    expanding_model = LinearRegression()
    expanding_model.fit(x_train, y_train)
    # Test and Prediction
    x_test = us_data[['xt_1']][i:i+1]
    y_pred = expanding_model.predict(x_test)[0][0]
    actual = us_data['Log Returns'].iloc[i]
    benchmark_prediction = np.mean(y_train)
    # Appending to lists
    expanding_model_prediction.append(y_pred)
    expanding_actual.append(actual)
    expanding_benchmark_prediction.append(benchmark_prediction)

expanding_mse_model = mean_squared_error(expanding_actual, expanding_model_prediction)
expanding_mse_benchmark = mean_squared_error(expanding_actual, expanding_benchmark_prediction)

print(expanding_mse_model, expanding_mse_benchmark)
print("The out of sample R squared is: ", 1 - (expanding_mse_model/expanding_mse_benchmark))

delta_rmse = np.sqrt(expanding_mse_benchmark) - np.sqrt(expanding_mse_model)
print("The Delta RMSE is: ", delta_rmse)

0.001945448402323024 0.0019370741194388087
The out of sample R squared is:  -0.004323160791927627
The Delta RMSE is:  -9.503331749566996e-05


This Implies that the Model is NOT better than the Benchmark.

# Rolling Window Regression

In [6]:
# Expanding Window Regression
rolling_model_prediction = []
rolling_benchmark_prediction = []
rolling_actual = []

for i in range(120, len(us_data) - 1, 1): #2784
    #setup data
    start = i - 120
    x_train = us_data[['xt_1']][start:i]
    y_train = us_data[['Log Returns']][start:i]
    # Run Regression
    rolling_model = LinearRegression()
    rolling_model.fit(x_train, y_train)
    # Test and Prediction
    x_test = us_data[['xt_1']][i:i+1]
    y_pred = rolling_model.predict(x_test)[0][0]
    actual = us_data['Log Returns'].iloc[i]
    benchmark_prediction = np.mean(y_train)
    # Appending to lists
    rolling_model_prediction.append(y_pred)
    rolling_actual.append(actual)
    rolling_benchmark_prediction.append(benchmark_prediction)

rolling_mse_model = mean_squared_error(rolling_actual, rolling_model_prediction)
rolling_mse_benchmark = mean_squared_error(rolling_actual, rolling_benchmark_prediction)

print(rolling_mse_model, rolling_mse_benchmark)
print("The out of sample R squared is: ", 1 - (rolling_mse_model/rolling_mse_benchmark))

delta_rmse = np.sqrt(rolling_mse_benchmark) - np.sqrt(rolling_mse_model)
print("The Delta RMSE is: ", delta_rmse)

0.0020666039279573947 0.0019483483741248551
The out of sample R squared is:  -0.0606952819131521
The Delta RMSE is:  -0.0013198162279665707


This implies that the Rolling Window Regression Model also does not outperform the Benchmark Model. 

# Expanding Model Clark and West Test

In [10]:
# Clark and West Test
f_i = []
for i in range(len(expanding_actual)):
    y_actual = expanding_actual[i]
    y_benchmark_pred = expanding_benchmark_prediction[i]
    y_competing_pred = expanding_model_prediction[i]

    # Clark and West test correction term
    correction_term = (y_actual - y_competing_pred)**2 - (y_benchmark_pred - y_competing_pred)**2
    f_i.append((y_actual - y_benchmark_pred)**2 - correction_term)

Y = np.array(f_i)
X = sm.add_constant(np.ones(len(Y)))
# Fit the regression model
model = sm.OLS(Y, X).fit()
# Print the estimated constant term (intercept)
print(f"Estimated rho_hat: {model.params[0]}")

rho_hat = model.params[0]
se_rho_hat = model.bse[0]
CW = rho_hat / se_rho_hat
print("The CW Stat is: ", CW)    
p_value = scipy.stats.norm.sf(CW)
print("The P Value is:", p_value)

Estimated rho_hat: -5.087598333120338e-06
The CW Stat is:  -0.7830584157388782
The P Value is: 0.7832035953701767


Cannot reject the Null Hypothesis that MSE benchmark = MSE model

# Rolling Model Clark and West Test

In [11]:
# Clark and West Test
f_i = []
for i in range(len(rolling_actual)):
    y_actual = rolling_actual[i]
    y_benchmark_pred = rolling_benchmark_prediction[i]
    y_competing_pred = rolling_model_prediction[i]

    # Clark and West test correction term
    correction_term = (y_actual - y_competing_pred)**2 - (y_benchmark_pred - y_competing_pred)**2
    f_i.append((y_actual - y_benchmark_pred)**2 - correction_term)

Y = np.array(f_i)
X = sm.add_constant(np.ones(len(Y)))
# Fit the regression model
model = sm.OLS(Y, X).fit()
# Print the estimated constant term (intercept)
print(f"Estimated p (intercept): {model.params[0]}")

rho_hat = model.params[0]
se_rho_hat = model.bse[0]
CW = rho_hat / se_rho_hat
print("The CW Stat is: ", CW)    
p_value = scipy.stats.norm.sf(CW)
print("The P Value is:", p_value)

Estimated p (intercept): -3.304335868527804e-06
The CW Stat is:  -0.11451173595005465
The P Value is: 0.545583928173363


Cannot reject the Null Hypothesis that MSE benchmark is = MSE model.

# Expanding Model Market Timing Ability

In [153]:
# Expanding Regression Timing Ability
expanding_df = pd.DataFrame(
    {
        'actual return':expanding_actual,
        'forecast return':expanding_model_prediction
    }
)
expanding_df['market direction'] = expanding_df['actual return'].apply(lambda x: 1 if x > 0 else 0)
expanding_df['forecast direction'] = expanding_df['forecast return'].apply(lambda x: 1 if x > 0 else 0)
expanding_df['correct forecast'] = expanding_df['market direction'] == expanding_df['forecast direction']
c = expanding_df['correct forecast'].sum()
n = len(expanding_df)
p = c/n
print("P:", p)
z_score = (p - 0.5) / np.sqrt(p * (1 - p) / n)
print("Z Score:", z_score)

P: 0.5155839279008637
Z Score: 1.609176035148589


Z Score < 1.65 hence cannot reject the Null Hypothesis. No Market Timing Ability. 

# Rolling Model Market Timing Ability

In [154]:
# rolling
rolling_df = pd.DataFrame(
    {
        'actual return':rolling_actual,
        'forecast return':rolling_model_prediction
    }
)
rolling_df['market direction'] = rolling_df['actual return'].apply(lambda x: 1 if x > 0 else 0)
rolling_df['forecast direction'] = rolling_df['forecast return'].apply(lambda x: 1 if x > 0 else 0)
rolling_df['correct forecast'] = rolling_df['market direction'] == rolling_df['forecast direction']
c = rolling_df['correct forecast'].sum()
n = len(rolling_df)
p = c/n
print("P:", p)
z_score = (p - 0.5) / np.sqrt(p * (1 - p) / n)
print("Z Score:", z_score)

P: 0.5336087119789711
Z Score: 3.476568426313029


Z Score > 1.65. Hence we reject the Null Hypothesis. The model does have predictive ability. 